## Домашняя работа по теме "Снижение размерности данных"

__1__ Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
X, y = load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((105, 4), (45, 4))

Для обучения будем использовать модель стохастического градиентного бустинга. Понижать размерность данных пока не будем.

In [4]:
def gb_predict(X, trees_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании
    # прибавляются с шагом eta
    

    predictions = np.array(
        sum([eta * alg.predict(X) for alg in trees_list])
    )

    return predictions

In [5]:
def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)

In [6]:
def residual(y, z):
    return - (z - y)

In [7]:
def sgb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, eta, sample_coef=0.5):
    
    n_samples = X_test.shape[0]
    
    # Деревья будем записывать в список
    trees = []
    
    # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
    train_errors = []
    test_errors = []
    
    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
        indices = np.random.randint(0, n_samples, int(n_samples * sample_coef))
        X_train_sampled, y_train_sampled = X_train[indices], y_train[indices]

        # первый алгоритм просто обучаем на выборке и добавляем в список
        if len(trees) == 0:
            # обучаем первое дерево на обучающей выборке
            tree.fit(X_train_sampled, y_train_sampled)
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, eta)))
        else:
            # Получим ответы на текущей композиции
            target = gb_predict(X_train_sampled, trees, eta)
            
            # алгоритмы начиная со второго обучаем на сдвиг
            tree.fit(X_train_sampled, residual(y_train_sampled, target))
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, eta)))

        trees.append(tree)
        
    return trees, train_errors, test_errors

In [8]:
n_trees = 3
eta = 0.1
max_depth = 2

In [9]:
trees, train_errors_sgb, test_errors_sgb = sgb_fit(n_trees, max_depth, 
                                               X_train, X_test, 
                                               y_train, y_test, 
                                               eta, sample_coef=0.5)

In [10]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, eta):
    train_prediction = gb_predict(X_train, trees, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тренировочной выборке: {mean_squared_error(y_train, train_prediction)}')

    test_prediction = gb_predict(X_test, trees, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тестовой выборке: {mean_squared_error(y_test, test_prediction)}')

In [11]:
evaluate_alg(X_train, X_test, y_train, y_test, trees, eta)

Ошибка алгоритма из 3 деревьев глубиной 2     с шагом 0.1 на тренировочной выборке: 0.8651057619047622
Ошибка алгоритма из 3 деревьев глубиной 2     с шагом 0.1 на тестовой выборке: 0.8903436444444444


Теперь понизим размерность данных с помощью метода главных компонент

In [12]:
# Стандартизируем признаки

def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res


X = standard_scale(X)

In [13]:
# Найдем собственные значения и собственные вектора матрицы признаков
eig_val, eig_vecs = np.linalg.eig(X.T @ X)
display(eig_val, eig_vecs)

array([437.77467248, 137.10457072,  22.01353134,   3.10722546])

array([[ 0.52106591, -0.37741762, -0.71956635,  0.26128628],
       [-0.26934744, -0.92329566,  0.24438178, -0.12350962],
       [ 0.5804131 , -0.02449161,  0.14212637, -0.80144925],
       [ 0.56485654, -0.06694199,  0.63427274,  0.52359713]])

In [14]:

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_val[i]), eig_vecs[:, i]) for i in range(len(eig_val))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979865, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.1045707202106, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.013531335697223, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.1072254642929384, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [15]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [16]:
# Сформируем новую матрицу "объекты-признаки"
Z = X.dot(W)
Z

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356],
       [-2.1663101 , -1.04369065],
       [-2.32613087, -0.13307834],
       [-2.2184509 ,  0.72867617],
       [-2.6331007 ,  0.96150673],
       [-2.1987406 , -1.86005711],
       [-2.26221453, -2.68628449],
       [-2.2075877 , -1.48360936],
       [-2.19034951, -0.48883832],
       [-1.898572  , -1.40501879],
       [-2.34336905, -1.12784938],
       [-1.914323  , -0.40885571],
       [-2.20701284, -0.92412143],
       [-2.7743447 , -0.45834367],
       [-1.81866953, -0.08555853],
       [-2.22716331, -0.13725446],
       [-1.95184633,  0.62561859],
       [-2.05115137, -0.24216355],
       [-2.16857717, -0.52714953],
       [-2.13956345,

In [17]:
# разобъем снова на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(Z, y,
                                                    test_size=0.3,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((105, 2), (45, 2))

In [18]:
trees, train_errors_sgb, test_errors_sgb = sgb_fit(n_trees, max_depth, 
                                               X_train, X_test, 
                                               y_train, y_test, 
                                               eta, sample_coef=0.5)

In [19]:
evaluate_alg(X_train, X_test, y_train, y_test, trees, eta)

Ошибка алгоритма из 3 деревьев глубиной 2     с шагом 0.1 на тренировочной выборке: 0.8884018703703704
Ошибка алгоритма из 3 деревьев глубиной 2     с шагом 0.1 на тестовой выборке: 0.9065751697530859


__Вывод:__ после понижения размерности данных удалось повысить качество модели и уменьшить ошибку. 

__2__ *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [20]:
def pca_method(X, n):
    """Функция для понижения размерности
    с помощью метода главных компонент
    
    X: исходные данные
    n: желаемое количество признаков на выходе"""
    
    # разложение SVD
    U, s, V = np.linalg.svd(X)
    
    # собственные векторы матрицы
    vecs = V.T
    
    # собственные значения
    vals = np.square(s)
    
    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(vals[i]), vecs[:, i]) for i in range(len(vals))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    
    # Сформируем вектор весов из собственных векторов, соответствующих первым n главным компонентам
    W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(n)])
    
    # Сформируем новую матрицу "объекты-признаки"
    Z = X.dot(W)
    
    return Z

In [21]:
Z2 = pca_method(X, 2)

In [22]:
# разобъем снова на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(Z2, y,
                                                    test_size=0.3,
                                                    random_state=1,
                                                    stratify=y)
X_train.shape, X_test.shape

((105, 2), (45, 2))

In [23]:
trees, train_errors_sgb, test_errors_sgb = sgb_fit(n_trees, max_depth, 
                                               X_train, X_test, 
                                               y_train, y_test, 
                                               eta, sample_coef=0.5)

In [24]:
evaluate_alg(X_train, X_test, y_train, y_test, trees, eta)

Ошибка алгоритма из 3 деревьев глубиной 2     с шагом 0.1 на тренировочной выборке: 0.8970083295238097
Ошибка алгоритма из 3 деревьев глубиной 2     с шагом 0.1 на тестовой выборке: 0.9424904577777775
